# Exercise 1 - Logistic Regression

In [36]:
import tensorflow as tf
from tensorflow.keras import utils

## Objectives

In this exercise we will implement the following functions:
* `softmax`: computes the softmax (normalised exponential function) of a input tensor;
* `cross_entropy`: calculates the cross-entropy loss between one-hot encoded prediction and ground truth vectors;
* `model`: logistic regression algorithm;
* `accuracy`: calculates the accuracy between a set of predictions and corresponding ground truth labels.

## 1. Introduction

In [2]:
### Importing required modules

In [3]:
import numpy as np
import os

In [4]:
### Setting environment variables

In [5]:
ENV_COLAB = False                # True if running in Google Colab instance

In [6]:
# Root directory
DIR_BASE = '' if not ENV_COLAB else '/content/'

In [7]:
# Subdirectory to save output files
DIR_OUT = os.path.join(DIR_BASE, 'out/')
# Subdirectory pointing to input data
DIR_SRC = os.path.join(DIR_BASE, 'data/')

### 1.1. Softmax

The [softmax function](https://en.wikipedia.org/wiki/Softmax_function) is a generalisation of the sigmoid [logistic function](https://en.wikipedia.org/wiki/Logistic_function) to multiple dimensions. In machine learning, particularly for logistic regression, the softmax function $\phi$ acts as a decision boundary applied to multi-class datasets, computing the probability of each observation $x_{i}$ belonging to one of $j = i,...,k$ class labels (assuming an independent relationship between the classes),

$$
\begin{align}
    P\left(y=j \ \vert \ z_{i}\right) = \phi_{softmax}\left(z_{i}\right) 
    = \frac{\mathcal{e}^{z_{i}}}{\sum_{j=0}^{k}\mathcal{e}^{z_{k}^{i}}}.
    \end{align}
$$
The input $z$ is defined to be
$$
\begin{align}
    z &= w_{0}x_{0} + w_{1}x_{1} + \ldots + w_{m}x_{m} = \sum_{i=0}^{m} w_{i}x_{i} = \mathrm{w}^{\top}\mathrm{x}.
    \end{align}
$$
such that $\mathrm{w}$ is the weight vector, $\mathrm{x}$ is the feature vector belonging to a single training observation, and $w_{0}$ is the bias unit.

The softmax function computes the probability for each class $P\left(y=j \vert x_{i}; w_{j}\right)$, then a correction step is applied to the predictions during training using a cost function that minimises the cross-entropy over the training set observations.

### 1.2. Cross-entropy

In machine learning, [cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_loss_function_and_logistic_regression) is often used as a loss function computed between two probability distributions. Given a set of predictions $q_{i}$ and corresponding true probability values $p_{i}$ we can compute the cross-entropy loss, i.e., _log loss_ [1],

$$
\begin{align}
    H\left(p,q\right) = -\sum_{i}{p_i}\mathrm{log}q_i = -ylog\hat{y} - \left(1-y\right)\mathrm{log}\left(1-\hat{y}\right)
    \end{align}
$$

which serves as a measure of dissimilarity between $p$ and $q$.

For a set of $n = 1,\ldots,N$ training observations, we can compute the average of the loss function over all observations such that $H\left(p,q\right)$ becomes

$$
\begin{align}
    J(\mathrm{w}) = \frac{1}{N}\sum_{n=1}^{N}H\left(p_{n},q_{n}\right) 
    = -\frac{1}{N}\sum_{n=1}^{N} \begin{bmatrix} y_{n}\mathrm{log}\hat{y}_{n} + \left(1-y_{n}\right)\mathrm{log}\left(1-\hat{y}_{n}\right) \end{bmatrix}.
    \end{align}
$$

#### Cross-entropy loss function

The cross-entropy loss function is defined for a training sample $x_{i}$ belonging to class $j$ as

$$
\begin{align}
    loss\left(x, y; w\right) &= H\left(y, \hat{y}\right) = \sum_{j} y_{j}\mathrm{log}\hat{y}_{j} = -\mathrm{log}\frac{\mathcal{e}^{w_{j}^{\top}x_{i}}}{\sum_{j=1}^{k} \mathcal{e}^{w_{j}^{\top}x_{i}}}
    \end{align}
$$
where $y$ denotes the [one-hot](https://en.wikipedia.org/wiki/One-hot) encoded vector and $\hat{y}$ denotes the probability distribution $h\left(x_{i}\right)$.

The cross-entropy loss function for all observations $\left(\mathrm{X}_{i}, \mathrm{Y}_{i}\right)_{i=1}^{N}$ is then

$$
\begin{align}
    loss\left(\mathrm{X}, \mathrm{Y}; \mathrm{w}\right) = -\sum_{i=1}^{N}\sum_{j=1}^{k} I\left[y_{i} = j\right]\mathrm{log}\frac{\mathcal{e}^{w_{j}^{\top}x_{i}}}{\sum_{j=1}^{k}\mathcal{e}^{w_{j}^{\top} x_{i}}}
    \end{align}
$$

## 2. Programming Task

### 2.1. Softmax

In this exercise, you have to implement 4 different functions:

* `softmax`: compute the softmax of a vector. This function takes as input a tensor and outputs a discrete probability distribution.

In [8]:
### From Udacity's `logistic.py`

In [9]:
def softmax(logits):
    """
    softmax implementation
    args:
    - logits [tensor]: 1xN logits tensor
    returns:
    - soft_logits [tensor]: softmax of logits
    """
    # IMPLEMENT THIS FUNCTION
    
    soft_logits = np.exp(logits) / np.sum(np.exp(logits))
    return soft_logits

In [45]:
### Testing the softmax function with N=7 predictions
x = [1.0, 2.0, 3.0, 1.0, 2.0, 2.0, 3.0]
x_scaled = softmax(x)
x_scaled

array([0.04010756, 0.10902364, 0.29635698, 0.04010756, 0.10902364,
       0.10902364, 0.29635698])

### 2.2. Cross-entropy

* `cross_entropy`: calculate the cross entropy loss given a vector of predictions (after softmax) and a vector of ground truth (one-hot vector).

In [30]:
### From Udacity's `logistic.py`

In [31]:
def cross_entropy(scaled_logits, one_hot):
    """
    Cross entropy loss implementation
    args:
    - scaled_logits [tensor]: NxC tensor where N batch size / C number of classes
    - one_hot [tensor]: one hot tensor
    returns:
    - loss [tensor]: cross entropy 
    """
    # IMPLEMENT THIS FUNCTION
    
    n_classes = one_hot.shape[0]
    class_label = one_hot.argmax(axis=1)
    log_likelihood = -np.log(scaled_logits[:n_classes, class_label])
    loss = np.sum(log_likelihood) / n_classes
    return loss

In [43]:
# Creating our ground-truth labels and using one-hot encoding
y = np.array([2.0, 2.0, 3.0, 0.0, 2.0, 1.0, 3.0])
y_one_hot = utils.to_categorical(y, num_classes=np.unique(y).shape[0])
y_one_hot

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [47]:
### Creating pseudo-batched prediction data by repeating 'predictions'
X_scaled = np.stack([x_scaled] * 3)
X_scaled

array([[0.04010756, 0.10902364, 0.29635698, 0.04010756, 0.10902364,
        0.10902364, 0.29635698],
       [0.04010756, 0.10902364, 0.29635698, 0.04010756, 0.10902364,
        0.10902364, 0.29635698],
       [0.04010756, 0.10902364, 0.29635698, 0.04010756, 0.10902364,
        0.10902364, 0.29635698]])

In [50]:
### Testing the cross-entropy loss function with N=1 batch
loss = cross_entropy(X_scaled, y_one_hot)
loss

6.648571590055648

### 2.3. Logistic Regression model

* `model`: takes a batch of images (stack of images along the first dimensions) and feeds it through the logistic regression model.

In [ ]:
### From Udacity's `logistic.py`

In [ ]:
def model(X, W, b):
    """
    logistic regression model
    args:
    - X [tensor]: input HxWx3
    - W [tensor]: weights
    - b [tensor]: bias
    returns:
    - output [tensor]
    """
    # IMPLEMENT THIS FUNCTION
    return 

### 2.4. Prediction accuracy

* `accuracy`: given a vector of predictions and a vector of ground truth, calculate the accuracy.

In [ ]:
### From Udacity's `logistic.py`

In [ ]:
def accuracy(y_hat, Y):
    """
    calculate accuracy
    args:
    - y_hat [tensor]: NxC tensor of models predictions
    - y [tensor]: N tensor of ground truth classes
    returns:
    - acc [tensor]: accuracy
    """
    # IMPLEMENT THIS FUNCTION
    return acc

## Tips

You can leverage the `tf.boolean_mask` function to calculate the cross entropy. Keep in mind
that most elements of the ground truth vector are zeros.

## Credits
This assignment was prepared by Thomas Hossler and Michael Virgo et al., Winter 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).

References
* [1] Ji, S. Xie, Y. Logistic Regression: From Binary to Multi-Class. http://people.tamu.edu/~sji/classes/LR.pdf


Helpful resources:
* [Softmax Regression and How is it Related to Logistic Regression? | KDnuggets](https://www.kdnuggets.com/2016/07/softmax-regression-related-logistic-regression.html)
* [Softmax and Cross Entropy Loss | P. Dahal](https://deepnotes.io/softmax-crossentropy)